In [ ]:
# transformers와 torch 라이브러리를 임포트합니다.
# transformers는 Hugging Face에서 제공하는 다양한 사전 학습된 모델과 도구를 제공하며,
# torch는 딥러닝을 위한 PyTorch 라이브러리입니다.
import transformers==4.47.1
import torch==2.5.1+cu121

In [ ]:
# 사용할 모델의 ID를 설정합니다.
# 여기서는 한국어 Llama 모델인 "MLP-KTLim/llama-3-Korean-Bllossom-8B"를 실습에 사용합니다.
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

In [ ]:
# 텍스트 생성 파이프라인을 설정합니다.
pipeline = transformers.pipeline(
    "text-generation",  # 작업 유형을 'text-generation'으로 설정하여 텍스트 생성 기능을 사용합니다.
    model=model_id,  # 지정한 모델 ID로 모델을 로드합니다.
    model_kwargs={"torch_dtype": torch.bfloat16},  # 모델을 bfloat16 데이터 타입으로 설정하여 메모리 효율을 높입니다.
    device_map="auto",  # 사용 가능한 GPU를 자동으로 할당하여 실행합니다.
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# 모델을 평가 모드로 전환합니다.
# 평가 모드에서는 훈련 기능이 비활성화되며, 모델이 추론(예측)에 최적화된 상태로 변경됩니다.
pipeline.model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [ ]:
# AI 어시스턴트가 사용자 질문에 대해 친절하게 답변할 수 있도록
# 대화 형식의 프롬프트를 설정하고, 필요한 종료 조건을 정의하는 과정입니다.
# PROMPT와 instruction을 통해 AI의 역할과 사용자 요청을 설정하고,
# messages 리스트로 대화 흐름을 정의하여 최종적으로 prompt를 생성합니다.
# terminators 리스트는 대화 종료에 사용될 토큰 ID를 지정하여,
# AI가 적절한 시점에 대화를 마칠 수 있도록 합니다.

# AI 어시스턴트의 역할을 정의하는 문구를 PROMPT 변수에 저장합니다.
PROMPT = '''당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''

# 사용자가 AI에게 요청하는 작업 내용을 instruction 변수에 저장합니다.
instruction = "서울의 유명한 관광 코스를 만들어줄래?"

# messages 리스트를 생성하여 대화의 흐름을 정의합니다.
messages = [
    {"role": "system", "content": f"{PROMPT}"},  # "system" 역할로 AI의 기본 역할을 정의하는 PROMPT를 포함합니다.
    {"role": "user", "content": f"{instruction}"}  # "user" 역할로 사용자가 요청한 instruction을 포함합니다.
]

In [ ]:
# 메시지 리스트를 기반으로 대화용 prompt를 생성합니다.
# pipeline.tokenizer.apply_chat_template 함수를 사용하여 messages 리스트를 템플릿 형식에 맞게 변환하고,
# tokenize=False로 설정해 텍스트 그대로 생성에 활용할 수 있도록 합니다.
prompt = pipeline.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True  # 텍스트 생성을 위한 추가 정보를 포함해 prompt를 구성합니다.
)


In [ ]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

서울의 유명한 관광 코스를 만들어줄래?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [ ]:
# 대화를 종료하는 데 사용될 토큰 ID들이 포함된 terminators 리스트를 정의합니다.
# 이 리스트는 종료 조건으로 사용되는 특별 토큰(eos_token_id와 <|eot_id|>)을 포함하여 대화가 적절히 종료될 수 있도록 합니다.
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
# outputs 변수에 텍스트 생성 결과를 저장합니다. pipeline 함수는 생성된 텍스트를 반환합니다.
outputs = pipeline(
    prompt,  # 앞서 생성된 prompt를 사용하여 모델이 텍스트를 생성하도록 합니다.
    max_new_tokens=2048,  # 최대 2048개의 새로운 토큰을 생성할 수 있도록 설정합니다.
    eos_token_id=terminators,  # terminators 리스트에 포함된 종료 토큰을 만나면 생성을 중단합니다.
    do_sample=True,  # 샘플링 방식으로 텍스트를 생성하여 결과가 다양하게 나올 수 있게 합니다.
    temperature=0.6,  # 샘플링 확률 분포를 조절하여 모델이 결정적으로 또는 창의적으로 행동하도록 설정합니다.
    top_p=0.9  # 상위 90%에 해당하는 누적 확률의 토큰만을 고려하여 샘플링을 진행합니다.
)

# 생성된 텍스트 출력. prompt 길이 이후의 텍스트만 출력하여 사용자 요청에 대한 응답을 표시합니다.
print(outputs[0]["generated_text"][len(prompt):])

서울은 다양한 문화, 역사, 자연, 그리고 현대적인 매력을 겸비한 도시로, 많은 관광객들이 찾는 인기 있는 장소가 많습니다. 여기 서울의 유명한 관광 코스를 소개합니다.

### 코스 1: 역사와 문화의 거리

1. **경복궁**
   - 서울의 대표적인 조선 왕조의 궁궐로, 경복궁은 조선 시대의 궁궐 건축과 정원 문화를 체험할 수 있는 곳입니다.

2. **창덕궁**
   - 경복궁과 함께 서울의 두 번째 큰 궁궐로, 수문장 부사관과 인화문이 유명합니다.

3. **북촌 한옥마을**
   - 전통 한옥이 잘 보존된 마을로, 조선 시대의 전통 건축물을 직접 보며 전통 생활을 체험할 수 있습니다.

4. **인사동**
   - 전통 문화와 현대 예술이 조화를 이루는 거리로, 전통 한옥 건물과 다양한 갤러리, 상점이 즐비되어 있습니다.

### 코스 2: 현대와 자연의 만남

1. **남산 서울타워**
   - 서울의 중심부에 위치한 전망대로, 서울 전경을 한눈에 볼 수 있습니다. 특히 밤에는 서울의 야경이 아름답습니다.

2. **한강공원**
   - 한강변을 따라 위치한 공원으로, 자전거 도로, 산책로, 공연장 등 다양한 시설이 갖추어져 있습니다. 특히 여름에는 한강 수영과 자전거 타기 등 다양한 여가 활동이 가능합니다.

3. **동대문**
   - 서울의 상권과 문화의 중심지로, 동대문 디자인 플라자(DDP)와 동대문 시장, 영화관 등이 있습니다.

4. **홍대**
   - 젊음과 문화가 융성하는 지역으로, 다양한 카페, 레스토랑, 클럽, 갤러리 등이 모여 있습니다.

### 코스 3: 쇼핑과 먹거리는 한꺼번에

1. **명동**
   - 서울의 대표적인 쇼핑 거리로, 다양한 패션 브랜드와 전통 음식점이 모여 있습니다.

2. **여의도**
   - 국제적인 분위기의 쇼핑몰과 관광 명소가 즐비한 곳으로, 여의도국립공원과 여의도 세종문화회관 등도 있습니다.

3. **광장시장**
   - 전통 시장으로, 다양한 음식과 전통 용품을 구입할 수 있습니다. 특히

# 프롬프트 직접 만들어서 바로 쓰기

In [ ]:
prompt = "서울에서 갈만한 곳 추천해줘"

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

print(outputs[0]["generated_text"][len(prompt):])

요! 서울은 다양한 문화와 역사, 자연이 어우러진 도시입니다. 서울을 방문하면 꼭 경험해야 할 곳들을 몇 가지 추천해 드릴게요.

1. **경복궁**: 조선 왕조의 첫 궁궐로, 한국의 역사와 문화를 체험할 수 있는 곳입니다. 특히, 정침(정문)과 경회루는 꼭 방문해야 할 곳입니다.

2. **남산서울타워(Namsan Seoul Tower)**: 서울의 대표적인 관광 명소 중 하나로, 서울 전경을 한눈에 볼 수 있는 전망대입니다. 특히, 밤에는 서울의 야경이 아름다워요.

3. **홍대**: 젊은이들이 모이는 거리로, 다양한 카페, 레스토랑, 클럽, 갤러리 등이 있습니다. 홍대는 서울의 문화와 예술을 느낄 수 있는 곳입니다.

4. **명동**: 쇼핑과 식사, 엔터테인먼트가 모두 가능한 거리로, 다양한 브랜드의 매장이 몰려 있습니다. 명동은 서울의 대표적인 쇼핑 거리로, 매일 많은 관광객들이 찾아요.

5. **한
